# Kaggle Dataset - 'https://www.kaggle.com/datasets/sanidhyak/human-face-emotions/data'

In [ ]:
!pip install split-folders


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

import splitfolders


In [ ]:
INPUT_DIR = "/kaggle/input/human-face-emotions/data"


In [ ]:
OUTPUT_DIR = "/kaggle/working/emotion_split"

# Split: 70% train, 15% val, 15% test
splitfolders.ratio(
    INPUT_DIR,
    output=OUTPUT_DIR,
    seed=42,
    ratio=(0.7, 0.15, 0.15),
    group_prefix=None
)

print("✅ Dataset split completed!")


In [ ]:
TRAIN_DIR = os.path.join(OUTPUT_DIR, "train")
VAL_DIR   = os.path.join(OUTPUT_DIR, "val")
TEST_DIR  = os.path.join(OUTPUT_DIR, "test")

emotion_classes = [
    d for d in os.listdir(TRAIN_DIR)
    if os.path.isdir(os.path.join(TRAIN_DIR, d))
]

print("Emotion Classes:", emotion_classes)


In [ ]:
IMG_SIZE = 224      # ✅ As you requested
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",              # ✅ COLOR
    batch_size=BATCH_SIZE,
    shuffle=True,
    classes=emotion_classes,
    class_mode="sparse"
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    shuffle=False,
    classes=emotion_classes,
    class_mode="sparse"
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    shuffle=False,
    classes=emotion_classes,
    class_mode="sparse"
)


In [ ]:
class_names = list(train_generator.class_indices.keys())
NUM_CLASSES = train_generator.num_classes

print("Classes:", class_names)
print("Number of classes:", NUM_CLASSES)
print("Class indices:", train_generator.class_indices)


In [ ]:
for images, labels in train_generator:
    plt.figure(figsize=(10, 5))
    for i in range(9):
        plt.subplot(3, 3, i + 1)

        img = images[i]
        plt.imshow(img)

        true_index = int(labels[i])
        true_label = class_names[true_index]

        plt.title(true_label)
        plt.axis("off")
    break

plt.show()


In [ ]:
model = Sequential([
    Input(shape=(224, 224, 3)),   # ✅ 224 RGB input

    Conv2D(32, (3,3), activation="relu"),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D(2,2),

    Conv2D(256, (3,3), activation="relu"),
    MaxPooling2D(2,2),

    Flatten(),

    Dense(256, activation="relu"),
    Dropout(0.5),

    Dense(NUM_CLASSES, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30
)


In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc*100:.2f}%")


In [ ]:
model.save("emotion_cnn_rgb_224.h5")
print("✅ 224×224 RGB emotion model saved!")


In [ ]:
# Install & Import Libraries

!pip install split-folders


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.applications import VGG16

import splitfolders


In [ ]:
# Split Dataset into Train / Val / Test

INPUT_DIR = "/kaggle/input/human-face-emotions/data"   # change to your path
OUTPUT_DIR = "/kaggle/working/emotion_split"

splitfolders.ratio(
    INPUT_DIR,
    output=OUTPUT_DIR,
    seed=42,
    ratio=(0.7, 0.15, 0.15)
)

print("✅ Dataset split completed!")


In [ ]:
# Define Paths & Load Classes

TRAIN_DIR = os.path.join(OUTPUT_DIR, "train")
VAL_DIR   = os.path.join(OUTPUT_DIR, "val")
TEST_DIR  = os.path.join(OUTPUT_DIR, "test")

emotion_classes = [
    d for d in os.listdir(TRAIN_DIR)
    if os.path.isdir(os.path.join(TRAIN_DIR, d))
]

print("Emotion Classes:", emotion_classes)


In [ ]:
# Data Generators (rescale = 1./255)

IMG_SIZE = 224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    shuffle=True,
    classes=emotion_classes,
    class_mode="sparse"
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    shuffle=False,
    classes=emotion_classes,
    class_mode="sparse"
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    shuffle=False,
    classes=emotion_classes,
    class_mode="sparse"
)


In [ ]:
class_names = list(train_generator.class_indices.keys())
NUM_CLASSES = train_generator.num_classes

print("Classes:", class_names)
print("Number of classes:", NUM_CLASSES)
print("Class indices:", train_generator.class_indices)


In [ ]:
# Visualize Training Samples (RGB)

for images, labels in train_generator:
    plt.figure(figsize=(10, 5))
    for i in range(9):
        plt.subplot(3, 3, i + 1)

        img = images[i]
        plt.imshow(img)

        true_index = int(labels[i])
        true_label = class_names[true_index]

        plt.title(true_label)
        plt.axis("off")
    break

plt.show()


In [ ]:
# Build VGG16 Model (rescale version)

base_model = VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze all VGG16 layers
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
x = base_model.output
x = Flatten()(x)

x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)

outputs = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


In [ ]:
# Train the Model

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20
)


In [ ]:
# Plot Accuracy & Loss

acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(acc, label="Train Accuracy")
plt.plot(val_acc, label="Val Accuracy")
plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(loss, label="Train Loss")
plt.plot(val_loss, label="Val Loss")
plt.title("Loss")
plt.legend()

plt.show()


In [ ]:
# Evaluate on Test Set

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc*100:.2f}%")


In [ ]:
# Predict ONE Validation Image (Like Your Earlier Code)

for images, labels in val_generator:
    img = images[0]

    true_index = int(labels[0])
    true_label = class_names[true_index]

    preds = model.predict(np.expand_dims(img, axis=0), verbose=0)[0]
    pred_index = np.argmax(preds)
    pred_label = class_names[pred_index]
    confidence = preds[pred_index] * 100

    plt.imshow(img)
    plt.axis("off")
    plt.title(
        f"Actual: {true_label}\n"
        f"Predicted: {pred_label}\n"
        f"Confidence: {confidence:.2f}%"
    )
    break


In [ ]:
# Multiple Predictions Visualization (3×3 Grid

plt.figure(figsize=(15, 15))

for images, labels in val_generator:
    for i in range(9):
        img = images[i]

        true_index = int(labels[i])
        true_label = class_names[true_index]

        preds = model.predict(np.expand_dims(img, axis=0), verbose=0)[0]
        pred_index = np.argmax(preds)
        pred_label = class_names[pred_index]
        confidence = preds[pred_index] * 100

        plt.subplot(3, 3, i + 1)
        plt.imshow(img)
        plt.title(
            f"A: {true_label}\n"
            f"P: {pred_label}\n"
            f"Conf: {confidence:.2f}%",
            fontsize=9
        )
        plt.axis("off")

    break

plt.show()


In [ ]:
# Save the Model

model.save("emotion_vgg16_sparse_model.h5")
print("✅ VGG16 emotion model saved successfully!")


In [ ]:
# Webcam

import cv2
import numpy as np
import tensorflow as tf

# ==========================
# Load Haar Cascade
# ==========================
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# ==========================
# Load Trained VGG16 Model
# ==========================
IMG_SIZE = 224
MODEL_PATH = "emotion_vgg16_sparse_model.h5"   # your VGG16 model

model = tf.keras.models.load_model(MODEL_PATH)
print("✅ VGG16 emotion model loaded")

# ==========================
# Class Names (MUST match training order!)
# ==========================
class_names = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]

# ==========================
# Start Webcam
# ==========================
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --------------------
    # Convert full frame to grayscale for face detection only
    # --------------------
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # --------------------
    # Detect faces on grayscale frame
    # --------------------
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(80, 80)
    )

    for (x, y, w, h) in faces:

        # --------------------
        # Crop face (COLOR, BGR)
        # --------------------
        face_color = frame[y:y+h, x:x+w]   # BGR crop

        if face_color.size == 0:
            continue

        # --------------------
        # Preprocess for VGG16 (rescale = 1./255 version)
        # --------------------
        # Resize
        face = cv2.resize(face_color, (IMG_SIZE, IMG_SIZE))

        # Convert BGR -> RGB
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)   # (224, 224, 3)

        # Normalize (SAME as training)
        face_array = face.astype("float32") / 255.0

        # Add batch dimension -> (1, 224, 224, 3)
        face_array = np.expand_dims(face_array, axis=0)

        # --------------------
        # Predict Emotion
        # --------------------
        preds = model.predict(face_array, verbose=0)[0]

        """
        preds shape = (NUM_CLASSES,)
        example -> [0.02, 0.01, 0.05, 0.78, 0.04, 0.06, 0.04]

        pred_index = argmax(preds)
        confidence = preds[pred_index] * 100
        label = class_names[pred_index]
        """

        pred_index = np.argmax(preds)
        confidence = preds[pred_index] * 100
        label = class_names[pred_index]

        text = f"{label} ({confidence:.2f}%)"

        # --------------------
        # Draw Results
        # --------------------
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        cv2.putText(
            frame,
            text,
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0, 255, 0),
            2
        )

    # --------------------
    # Show Frame
    # --------------------
    cv2.imshow("Emotion Detection (VGG16 RGB)", frame)

    # Press ESC to exit
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
